<a href="https://colab.research.google.com/github/Akhila981010/project/blob/main/twitter_train_and_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install emojis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install trax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 58.6 MB/s eta 0:00:00


In [ ]:
import tweepy
import json
#import geocoder
import pandas as pd
import numpy as np

import os 
import time
import emojis


import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import random as rnd                       # pseudo-random number generator

import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings


# import relevant libraries
import trax

# import trax.fastmath.numpy

import trax.fastmath.numpy as np


# import trax.layers
from trax import layers as tl


from trax.supervised import training




In [ ]:

# Load Twitter API secrets from an external JSON file
secrets = json.load(open(r'XXXXXXXXXXXXXXXXX/secrets.json'))
access_token = secrets['access_token']
access_token_secret = secrets['access_token_secret']
api_key = secrets['api_key']
api_secret = secrets['api_secret']



FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# authorize api handshake

auth = tweepy.OAuthHandler(api_key, api_secret)

auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)



NameError: ignored

# Download Training Data

In [ ]:
# downloads sample twitter dataset. uncomment the line below if running on a local machine.
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [ ]:
# select the set of positive and negative tweets
nltk_positive_tweets = twitter_samples.strings('positive_tweets.json')
nltk_negative_tweets = twitter_samples.strings('negative_tweets.json')



In [ ]:
sentiment140_tweets = pd.read_csv('sentiment140.csv' , encoding = "ISO-8859-1", names = ["target", "ids", "date", "flag", "user", "text"])

sentiment140_tweets.tail()

FileNotFoundError: ignored

In [ ]:

fmap = {4:1, 0:0}

sentiment140_tweets['target'] = sentiment140_tweets['target'] .map(fmap)

NameError: ignored

In [ ]:
sentiment140_tweets['target'].value_counts()

1    800000
0    800000
Name: target, dtype: int64

In [ ]:

sentiment140_pos_tweets = list(sentiment140_tweets[sentiment140_tweets['target'] == 1]['text'])

sentiment140_neg_tweets = list(sentiment140_tweets[sentiment140_tweets['target'] == 0]['text'])

In [ ]:

all_positive_tweets = nltk_positive_tweets + sentiment140_pos_tweets

all_negative_tweets = nltk_negative_tweets + sentiment140_neg_tweets

In [ ]:
print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))

Number of positive tweets:  805000
Number of negative tweets:  805000


In [ ]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[rnd.randint(0,805000)])

# print negative in red
print('\033[91m' + all_negative_tweets[rnd.randint(0,805000)])

@DanielMoroney yo, found myself relating there with his dating joke. So even if I had gne 2nd round I wouldnt be &quot;creepy&quot; interesting 
@sakixry I totally understand u! 


In [ ]:
# Split positive set into validation and training
val_pos   = all_positive_tweets[724500:] # generating validation set for positive tweets
train_pos  = all_positive_tweets[:724500]# generating training set for positive tweets

# Split negative set into validation and training
val_neg   = all_negative_tweets[724500:] # generating validation set for negative tweets
train_neg  = all_negative_tweets[:724500] # generating training set for nagative tweets

# Combine training data into one set
train_x = train_pos + train_neg 

# Combine validation data into one set
val_x  = val_pos + val_neg

# Set the labels for the training set (1 for positive, 0 for negative)
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

# Set the labels for the validation set (1 for positive, 0 for negative)
val_y  = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

print(f"Length of train_x:  {len(train_x)}")
print(f"Length of val_x:  {len(val_x)}")

Length of train_x:  1449000
Length of val_x:  161000


# Pre Process Tweets

In [ ]:

# Our selected sample. Complex enough to exemplify each step
tweet = all_positive_tweets[300]
print(tweet)


Stats for the day have arrived. 2 new followers and NO unfollowers :) via http://t.co/xxlXs6xYwe.


In [ ]:

# download the stopwords from NLTK
nltk.download('stopwords')

#Import the english stop words list from NLTK
stopwords_english = stopwords.words('english') 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def process_tweet(tweet):  

    # remove old style retweet text "RT"
    new_tweet = re.sub(r'^RT[\s]+', '', tweet)
    
    # decode emojis to text descriptions
    new_tweet = emojis.decode(new_tweet)

    # remove hyperlinks
    new_tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', new_tweet)
    new_tweet = re.sub(r'http\S+', '', new_tweet)

    # remove hashtags
    new_tweet = re.sub(r'#', '', new_tweet)
    
    # remove underscores
    new_tweet = re.sub(r'_', '', new_tweet)

    # remove all numbers
    new_tweet = re.sub(r'[0-9]', '', new_tweet)

    # remove usernames
    new_tweet = re.sub('@[^\s]+', '', new_tweet)
    
    # remove punctuation even in the middle of a string "in.the.middle"
    new_tweet = re.sub(r'[^\w\s]',' ', new_tweet)
    
    # convert text to lower-case
    #new_tweet = new_tweet.lower()

    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(new_tweet)

    tweets_clean = []

    for word in tweet_tokens: # Go through every word in your tokens list
#        if (word not in stopwords_english and  # remove stopwords
#            word not in string.punctuation):  # remove punctuation
#            tweets_clean.append(word)
        if (word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)




    # Instantiate stemming class
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [] 

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
    
    return tweets_stem


In [ ]:
tweet = all_positive_tweets[300]
print(f'Original tweet:     {tweet}')

processed_tweet = process_tweet(tweet)
print(f'Processed tweet:     {processed_tweet}')

Original tweet:     Stats for the day have arrived. 2 new followers and NO unfollowers :) via http://t.co/xxlXs6xYwe.
Processed tweet:     ['stat', 'for', 'the', 'day', 'have', 'arriv', 'new', 'follow', 'and', 'no', 'unfollow', 'via']


# Build the Vocabulary

In [ ]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val

# Include special tokens 
# started with pad, end of line and unk tokens
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

# Note that we build vocab using training data
for tweet in train_x: 
    processed_tweet = process_tweet(tweet)
    for word in processed_tweet:
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are", len(Vocab), '\n')

first_10 = {k: Vocab[k] for k in sorted(Vocab.keys())[:10]}
display(first_10)

Total words in vocab are 200415 



{'__</e>__': 1,
 '__PAD__': 0,
 '__UNK__': 2,
 'a': 34,
 'aa': 3852,
 'aaa': 12810,
 'aaaa': 9957,
 'aaaaaaaaah': 136831,
 'aaaaaahhh': 156494,
 'aaaaaariel': 155213}

In [ ]:
json.dump(Vocab, open( "Vocab.json", 'w' ) )

# Build a Tensor from a Tweet

In [ ]:

def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
        
    '''  

    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = []
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.
        word_ID = vocab_dict.get(word, unk_ID)

        
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID) 
    
    return tensor_l

In [ ]:
print("Actual tweet is\n", val_pos[1])
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[1], vocab_dict=Vocab))

Actual tweet is
 4 hours and counting 

Tensor of tweet:
 [647, 24, 2137]


# Build a Batch Generator

In [ ]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of posstive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     

    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1


        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index = neg_index + 1

        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take
        
        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:

            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0] * n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor +  pad_l

            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1] * n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0] * n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
        example_weights = np.ones_like(targets)
        
        # note we use yield and not return
        yield inputs, targets, example_weights


In [ ]:
# Set the random number generator for the shuffle procedure
rnd.seed(42) 

# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

# Create the validation data generator
def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[  105    81  1531   128   719   187   315  1385    82   187     5   300
      0     0     0     0     0     0     0     0     0     0     0     0]
 [  713    97  5028     9   105  1053  5704   245  9294     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [ 1437   154   684   558  1922    37   137 45879  1495   144     9 16853
   4639  3345 22509 15478  6016     9   105  3090  1655   279    85  6217]
 [ 8729   412   105   989    28   105  3510   167   418   433    28   514
     24  2252   245   105 16888   280    59     0     0     0     0     0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


# Build a Model

In [ ]:
def classifier(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):
        

    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis=1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      log_softmax_layer # log softmax layer
    )

    # return the model of type
    return model

In [ ]:
tmp_model = classifier()

In [ ]:
print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Embedding_200415_256
  Mean
  Dense_2
  LogSoftmax
]

# Train the Model

In [ ]:

batch_size = 16
rnd.seed(42)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.0001),
    n_steps_per_checkpoint=100,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

model = classifier()

In [ ]:
! pwd

/content


In [ ]:
output_dir = '~/content/model_adam0001_90562_9010_/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/root/content/model_adam0001_90562_9010_/


In [ ]:
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''

    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)

    # Return the training_loop, since it has the model.
    return training_loop

In [ ]:
training_loop = train_model(model, train_task, eval_task, 90562, output_dir_expand)


Step      1: Total number of trainable weights: 51306754
Step      1: Ran 1 train steps in 15.08 secs
Step      1: train CrossEntropyLoss |  0.69381428
Step      1: eval  CrossEntropyLoss |  0.69212425
Step      1: eval          Accuracy |  0.56250000

Step    100: Ran 99 train steps in 20.93 secs
Step    100: train CrossEntropyLoss |  0.69148350
Step    100: eval  CrossEntropyLoss |  0.69511348
Step    100: eval          Accuracy |  0.62500000

Step    200: Ran 100 train steps in 16.41 secs
Step    200: train CrossEntropyLoss |  0.68915695
Step    200: eval  CrossEntropyLoss |  0.69455999
Step    200: eval          Accuracy |  0.43750000

Step    300: Ran 100 train steps in 13.61 secs
Step    300: train CrossEntropyLoss |  0.68721557
Step    300: eval  CrossEntropyLoss |  0.68443823
Step    300: eval          Accuracy |  0.68750000

Step    400: Ran 100 train steps in 13.20 secs
Step    400: train CrossEntropyLoss |  0.68618089
Step    400: eval  CrossEntropyLoss |  0.70708752
Step  

# Test the model

In [ ]:
# Create a generator object
tmp_train_generator = train_generator(16)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 21) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [ ]:
# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


DeviceArray([[-2.2622967 , -0.10993886],
             [-4.0666504 , -0.0172832 ],
             [-1.890605  , -0.16367304],
             [-2.0663567 , -0.13541478],
             [-0.77469456, -0.6177513 ],
             [-0.9887702 , -0.46526915],
             [-4.560339  , -0.01051354],
             [-0.8533397 , -0.55510825],
             [-0.45352408, -1.0089135 ],
             [-0.5496769 , -0.86070526],
             [-0.74579984, -0.64312863],
             [-0.11520672, -2.218078  ],
             [-0.1414603 , -2.0256321 ],
             [-0.5003785 , -0.9321689 ],
             [-2.1311235 , -0.12636155],
             [-0.04255414, -3.1781788 ]], dtype=float32)

In [ ]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -2.2623	Pos log prob -0.1099	 is positive? True	 actual 1
Neg log prob -4.0667	Pos log prob -0.0173	 is positive? True	 actual 1
Neg log prob -1.8906	Pos log prob -0.1637	 is positive? True	 actual 1
Neg log prob -2.0664	Pos log prob -0.1354	 is positive? True	 actual 1
Neg log prob -0.7747	Pos log prob -0.6178	 is positive? True	 actual 1
Neg log prob -0.9888	Pos log prob -0.4653	 is positive? True	 actual 1
Neg log prob -4.5603	Pos log prob -0.0105	 is positive? True	 actual 1
Neg log prob -0.8533	Pos log prob -0.5551	 is positive? True	 actual 1
Neg log prob -0.4535	Pos log prob -1.0089	 is positive? False	 actual 0
Neg log prob -0.5497	Pos log prob -0.8607	 is positive? False	 actual 0
Neg log prob -0.7458	Pos log prob -0.6431	 is positive? True	 actual 0
Neg log prob -0.1152	Pos log prob -2.2181	 is positive? False	 actual 0
Neg log prob -0.1415	Pos log prob -2.0256	 is positive? False	 actual 0
Neg log prob -0.5004	Pos log prob -0.9322	 is positive? False	 actual 0
N

# Build Accuracy Function and Test on One Batch of Data

In [ ]:
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch, output_dim) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """

    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos =  preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = np.array(is_pos, dtype = np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = is_pos_int == y

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = np.array(correct, dtype = np.float32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = correct_float * y_weights

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # denominator.
    weighted_num_correct = np.sum(weighted_correct_float)
 
    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = sum_weights / weighted_num_correct 


    return accuracy, weighted_num_correct, sum_weights

In [ ]:
# test your function
tmp_val_generator = val_generator(64)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 120.75471496582031%
Weighted number of correct predictions 53.0; weighted number of total observations predicted 64


# Evaluate the Model

In [ ]:
def test_model(generator, model):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0

    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(preds=pred, y=targets, y_weights=example_weight)
        
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct / total_num_pred

    return accuracy

In [ ]:
# testing the accuracy of your model: this takes around 20 seconds
model = training_loop.eval_model

accuracy = test_model(test_generator(16), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.7852


# Getting Live Twitter Data

In [ ]:
usa = 23424977  

# grab top 50 trending topics for the USA
trending_topics = api.trends_place(usa)                          

topic_list = []

# loop though each trending topic and create a list
for k in range(len(trending_topics[0]['trends'])):                

    topic = trending_topics[0]['trends'][k]['name']               
    
    topic_list.append(topic)
    
    print(topic)


Taysom Hill
Geraldo
Jameis
Edward Norton
Sean Payton
#TransDayOfRemembrance
#fridaymorning
Mr. President
Rick Scott
#FreePcFriday
Megan
Ed Norton
Autism Speaks
#FridayThoughts
#FridayVibes
Trump Virus
Meek
Tucker
Trumped
Romney
Good Friday
"Trump"
Al Capone
Myles Garrett
Gary Owen
Mnuchin
Bob Bauer
Shots Fired
Transgender Day of Remembrance
Trump Vaccine
Mr. Norton
Dershowitz
Tory
Jamies
Bridgeport
Tina Snow
Morgan Wallen
Arenado
SAINt JHN
Jazmine Sullivan
Finally Friday
Rounders
Today is Friday
Sasse
Dolph
Chapter 12
Benoit Mandelbrot
Michigan and Minnesota
Susan
Pookie Loc


In [ ]:
program_start = time.time()            # time the program

max_tweets = 50        # number of tweets to grab for each topic
n = 50                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'    
    
columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

recent_tweets_data = pd.DataFrame(columns = columns)                           # create empty dataframe    
    

for k in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town
    
    run_start = time.time()                                       # start time of the run for each topic   

    topic = trending_topics[0]['trends'][k]['name']               # the trending topic or hashtag (1 out of 50)  
                                                        
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'recent' ,                    # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    recent_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                      # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'recent'
    
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,            # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        recent_tweets_topic.loc[len(recent_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    recent_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = recent_tweets_topic[0:n]                                                     # keep only the top n
                            
    recent_tweets_data = pd.concat([recent_tweets_data, top_n], ignore_index = True, sort = False)              # concatenate top n to final dataframe
    
    run_end = time.time()                                                              # stop the time for the run
    run_duration = round((run_end - run_start)/60, 2)                                  # calculate duration
    print(f'Time for {topic} : {run_duration} mins')                                   # print duration       
    

program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    


Time for Taysom Hill : 0.03 mins
Time for Geraldo : 0.03 mins
Time for Jameis : 0.03 mins
Time for Edward Norton : 0.03 mins
Time for Sean Payton : 0.03 mins
Time for #TransDayOfRemembrance : 0.03 mins
Time for #fridaymorning : 0.03 mins
Time for Mr. President : 0.03 mins
Time for Rick Scott : 0.03 mins
Time for #FreePcFriday : 0.03 mins
Time for Megan : 0.02 mins
Time for Ed Norton : 0.03 mins
Time for Autism Speaks : 0.03 mins
Time for #FridayThoughts : 0.03 mins
Time for #FridayVibes : 0.07 mins
Time for Trump Virus : 0.03 mins
Time for Meek : 0.03 mins
Time for Tucker : 0.0 mins
Time for Trumped : 0.03 mins
Time for Romney : 0.03 mins
Time for Good Friday : 0.03 mins
Time for "Trump" : 0.03 mins
Time for Al Capone : 0.03 mins
Time for Myles Garrett : 0.03 mins
Time for Gary Owen : 0.03 mins
Time for Mnuchin : 0.03 mins
Time for Bob Bauer : 0.03 mins
Time for Shots Fired : 0.03 mins
Time for Transgender Day of Remembrance : 0.03 mins
Time for Trump Vaccine : 0.03 mins
Time for Mr. N

In [ ]:
recent_tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,JoeyMorris122,,3256,0,When Taysom Hill finally becomes QB1 https://t...,[],[],Taysom Hill,recent
1,KDot_Flow,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
2,BRGIANTSKNICKS4,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
3,LC_w0ng,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
4,CraigWLearn,"Buffalo, NY",924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent


In [ ]:
len(recent_tweets_data)

2450

In [ ]:
program_start = time.time()            # time the program

max_tweets = 100        # number of tweets to grab for each topic
n = 100                 # top n number of retweets to keep

#assert (max_tweets * len(df) * 50) <= 100000 , 'Request Limit Assert Error: Too many tweets! \n \n Twitter has a 24 hour limit of 100,000 requests \n \n Try changing "max_tweets"'


columns = ['username', 'user_location', 'retweetcount', 'favorites', 'text',   # set up columns for dataframes
           'hashtags', 'emojis', 'trending_topic', 'search_type']          

pop_tweets_data = pd.DataFrame(columns = columns)                 # create empty dataframe
    

for q in range(len(trending_topics[0]['trends'])):                # loop though each trending topic for each town

    topic = trending_topics[0]['trends'][q]['name']               # the trending topic or hashtag (1 out of 50)  
                                                                           
                                                                            # grab tweets with Cursor
    tweets = tweepy.Cursor(api.search, q = topic,                           # search for each trending topic                                 
                     lang="en", result_type = 'popular' ,                   # most recent tweets in english
                      tweet_mode = 'extended').items(max_tweets)            # longer tweets and grab max_tweets number of tweets
    
    tweet_list = [tweet for tweet in tweets]                                # create list of tweets
                
    pop_tweets_topic = pd.DataFrame(columns = columns)         # create dataframe to put in current top tweets for this town and trending topic
        
    for tweet in tweet_list:                                   # loop through each tweet that was grabbed
        
        username = tweet.user.screen_name              # store username
        user_location = tweet.user.location            # store location of user
        retweetcount = tweet.retweet_count             # store retweet count
        favorites = tweet.favorite_count               # store favorite count
        hashtags = tweet.entities['hashtags']          # store hashtags
        search_type = 'popular'
        
        try:                              
            text = tweet.retweeted_status.full_text    # store text if it's a retweet
        
        except AttributeError: 
            text = tweet.full_text                     # store text if it's a regular tweet
            
        emoji = list(emojis.get(text))                      # get the emojis    
        
        curr_tweet = [username, user_location, retweetcount, favorites, text,      # store current tweet's data in a list soon to be a row
                                hashtags, emoji, topic, search_type]                             
        
        pop_tweets_topic.loc[len(pop_tweets_topic)] = curr_tweet                   # add current tweet data to dataframe for town and topic
                            
    pop_tweets_topic.sort_values(by=['retweetcount', 'favorites'], inplace = True, ascending = False)        # sort the retweet values highest first
                            
    top_n = pop_tweets_topic[0:n]                                                                  # keep only the top n
                            
    pop_tweets_data = pd.concat([pop_tweets_data, top_n], ignore_index = True, sort = False)       # concatenate top n to final dataframe
            
    
program_end = time.time()                                                               # stop the program time
program_duration = round(((program_end - program_start) / 60), 2)                       # calculate duration
print(f'\n \n ************************************************** \n \n TOTAL TIME TO GRAB TWEETS : \t {program_duration} mins')    # print duration as total time
print('\n **************************************************')    

    


 
 ************************************************** 
 
 TOTAL TIME TO GRAB TWEETS : 	 0.73 mins

 **************************************************


In [ ]:
pop_tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,AdamSchefter,New York,1009,8346,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,popular
1,SportsCenter,,951,12221,Taysom Hill will start Sunday for the Saints a...,[],[],Taysom Hill,popular
2,AdamSchefter,New York,669,4902,Jameis Winston will not be part of any offensi...,[],[],Taysom Hill,popular
3,RapSheet,New York,524,4456,Intrigue in New Orleans: The #Saints have give...,"[{'text': 'Saints', 'indices': [29, 36]}]",[],Taysom Hill,popular
4,FieldYates,"Boston, MA",158,1925,A note: Taysom Hill is eligible to be utilized...,[],[],Taysom Hill,popular


In [ ]:
len(pop_tweets_data)

1083

In [ ]:
pop_tweets_data['trending_topic'].value_counts()

"Trump"                           89
Tory                              75
Trump Vaccine                     52
Mr. President                     42
Trump Virus                       40
Susan                             39
Mnuchin                           39
Romney                            36
Tucker                            35
Megan                             33
#TransDayOfRemembrance            31
Today is Friday                   30
Sasse                             30
Jameis                            30
Transgender Day of Remembrance    30
Shots Fired                       29
Good Friday                       29
Geraldo                           29
Myles Garrett                     29
Taysom Hill                       29
Meek                              26
Rounders                          26
Rick Scott                        25
#FridayThoughts                   24
Sean Payton                       20
Bob Bauer                         19
Finally Friday                    15
T

In [ ]:
tweets_data = pd.concat([recent_tweets_data, pop_tweets_data], ignore_index=True)

tweets_data.head()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
0,JoeyMorris122,,3256,0,When Taysom Hill finally becomes QB1 https://t...,[],[],Taysom Hill,recent
1,KDot_Flow,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
2,BRGIANTSKNICKS4,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
3,LC_w0ng,,924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent
4,CraigWLearn,"Buffalo, NY",924,0,Saints’ QB Taysom Hill will start Sunday vs. t...,[],[],Taysom Hill,recent


In [ ]:
tweets_data.tail()

,username,user_location,retweetcount,favorites,text,hashtags,emojis,trending_topic,search_type
3528,yungmal__15,,48,194,I’m Smoking on pookie loc tonighttt 😭😭😂😂 https...,[],"[😭, 😂]",Pookie Loc,popular
3529,CHXPO,Bleveland 💸 LA,30,114,Smoking on Pookie Loc Tonight !,[],[],Pookie Loc,popular
3530,OGJOHNNY5,stunna island,12,13,"“SMOKIN ON POOKIE LOC TONIGHT""",[],[],Pookie Loc,popular
3531,WhoIsClayJames,,7,13,Man Gucci said we smoking that Pookie Loc pack...,"[{'text': 'VERZUZ', 'indices': [108, 115]}]",[😓],Pookie Loc,popular
3532,jkboxing,"London, United Kingdom",5,6,When Gucci Mane said “We smoking on pookie loc...,"[{'text': 'VERZUZ', 'indices': [85, 92]}]",[],Pookie Loc,popular


In [ ]:
len(tweets_data)

3533

# Use Models on Live Twitter Data

In [ ]:
# this is used to predict on your own sentnece
def predict(sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return sentiment, round(float(preds_probs[0, 0]),4), round(float(preds_probs[0, 1]),4)

In [ ]:


for i in range(len(tweets_data)):

    tweets_data.loc[i, 'sentiment'], tweets_data.loc[i, 'neg_prob'], tweets_data.loc[i, 'pos_prob'] = predict(tweets_data['text'].iloc[i])

    

NameError: ignored

In [ ]:
tweets_data.head(50)

In [ ]:
max_pos = max(tweets_data['pos_prob'])

pos_tweets  = tweets_data.loc[tweets_data['pos_prob'] == max_pos][['text']]

for p in pos_tweets['text']:
    print(p) 



In [ ]:
max_neg = max(tweets_data['neg_prob'])

neg_tweets  = tweets_data.loc[tweets_data['neg_prob'] == max_neg][['text']]

for n in neg_tweets['text']:
    print(n) 



In [ ]:
difficult = "Great!! I got a flat tire!!"

predict(difficult)

In [ ]:

pred_positive_tweets = tweets_data.sort_values(by=['pos_prob'], inplace = False, ascending = False)       
                            
top_positive = pred_positive_tweets[0:25]     

top_positive

In [ ]:

pred_negative_tweets = tweets_data.sort_values(by=['neg_prob'], inplace = False, ascending = False)        
                            
top_negative = pred_negative_tweets[0:25]     

top_negative